In [1]:
from astropy.table import Table
import scipy.spatial.distance as dist
from astropy import units as u
from astropy.cosmology import Planck13
from astropy.io import fits
from astropy.coordinates import SkyCoord
import sptpol_software as sps
from sptpol_software.util.tools import stat
from sptpol_software.observation import *
import sptpol_software.observation as obs
import sptpol_software as sps
import sptpol_software.observation.sky
from sptpol_software.util import files
import PIL
import pickle
from astropy import constants as ap_const
import scipy.constants as const
from scipy.integrate import quad
import matplotlib.pyplot as plt
import pandas as pd
import glob
import astropy as ap
import scipy as sp
import numpy as np
import os
import time 
import pdb
os.chdir("/home/mitchell/Documents/masters/masters/scripts/")
import cmb
import galaxy_pairs
#data_location = input("Enter Path Location of data")
os.chdir("/home/mitchell/Documents/masters/masters/data/")
cwd = os.getcwd()

import scipy.ndimage
cosmo = Planck13

pywtl packages could not be imported.  This may break plotting code.
FlatLambdaCDM(name="Planck15", H0=67.7 km / (Mpc s), Om0=0.307, Tcmb0=2.725 K, Neff=3.05, m_nu=[0.   0.   0.06] eV, Ob0=0.0486)


In [2]:
df = pd.read_pickle('DES_cat.pkl')


In [3]:
print("Computing Comoving Distances from Redshifts in Galaxy Catalogue ...")
start = time.time()
df['COMOVING'] = pd.Series(
    cosmo.comoving_distance(df['ZREDMAGIC']))
df['COMOVING_E'] = pd.Series(
    cosmo.comoving_distance(df['ZREDMAGIC_E']))

Computing Comoving Distances from Redshifts in Galaxy Catalogue ...


In [4]:
norm_ra = []
for i in range(len(df)):
    norm_ra.append(galaxy_pairs.normalize(df['RA'][i],-180,180))
df['NORM_RA'] = pd.Series(norm_ra)


In [5]:
ra_range = np.array([-32,32])
dec_range = np.array([-52,-67])


In [6]:
cut_df = df[((df.DEC < dec_range[0]) & (df.DEC > dec_range[1]))
            & ((df.NORM_RA > ra_range[0]) & (df.NORM_RA < ra_range[1]))]
cut_df = cut_df.reset_index(drop=True)


In [7]:
sky_locs = np.asarray([cut_df['NORM_RA'].as_matrix().T,cut_df['DEC'].as_matrix().T]).T

frame_tree = scipy.spatial.cKDTree(sky_locs, leafsize=100)

result_set = frame_tree.query_pairs(r=0.4)

result_list = list(result_set)

result_array = np.asarray(result_list)

cut_pairs_df = pd.DataFrame(data=result_array,columns = ['galaxy_index_1','galaxy_index_2'])

In [8]:
cut_pairs_df['galaxy_index_1'] = cut_pairs_df.galaxy_index_1.astype(int)
cut_pairs_df['galaxy_index_2'] = cut_pairs_df.galaxy_index_2.astype(int)

print("Num of Pairs Produced on Footprint = " + str(len(cut_pairs_df)))

Num of Pairs Produced on Footprint = 6021580


In [9]:
gal_1_comov = cut_df['COMOVING'].iloc[cut_pairs_df['galaxy_index_1']]
gal_2_comov = cut_df['COMOVING'].iloc[cut_pairs_df['galaxy_index_2']]
comov_sep = pd.Series(abs(gal_1_comov.values - gal_2_comov.values))

In [10]:
cut_pairs_df['SEP_LOS'] = comov_sep


min_los = 100/cosmo.h
max_los = 200/cosmo.h


cut_pairs_df = cut_pairs_df[(cut_pairs_df.SEP_LOS < max_los) & (cut_pairs_df.SEP_LOS > min_los)]

cut_pairs_df = cut_pairs_df.reset_index()
print("Num of Pairs after LOS Cuts = " + str(len(cut_pairs_df)))

Num of Pairs after LOS Cuts = 886848


In [11]:
gal_1_z = cut_df['ZREDMAGIC'].iloc[cut_pairs_df['galaxy_index_1']]
gal_2_z = cut_df['ZREDMAGIC'].iloc[cut_pairs_df['galaxy_index_2']]

print("Appending Redshifts of Pair to Dataframe...")
cut_pairs_df['Z1'] = pd.Series(gal_1_z.values)
cut_pairs_df['Z2'] = pd.Series(gal_2_z.values)

end = time.time()
print("Done in " + str(end-start) + " seconds")
start = end

print("Computing Average Redshift...")
avg_redshift = pd.Series(np.mean([gal_1_z,gal_2_z],axis=0))

end = time.time()
print("Done in " + str(end-start) + " seconds")
start = end

print("Computing Transverse Comoving Distance at this redshift...")
transverse_comoving = cosmo.comoving_transverse_distance(avg_redshift)

cut_pairs_df['COMOV_TRV'] = transverse_comoving

end = time.time()
print("Done in " + str(end-start) + " seconds")
start = end

print("Appending (RA,DEC) to Dataframe...")
cut_pairs_df['RA_1'] = pd.Series(cut_df['RA'].iloc[cut_pairs_df['galaxy_index_1']].values)
cut_pairs_df['RA_2'] = pd.Series(cut_df['RA'].iloc[cut_pairs_df['galaxy_index_2']].values)
cut_pairs_df['DEC_1'] = pd.Series(cut_df['DEC'].iloc[cut_pairs_df['galaxy_index_1']].values)
cut_pairs_df['DEC_2'] = pd.Series(cut_df['DEC'].iloc[cut_pairs_df['galaxy_index_2']].values)

end = time.time()
print("Done in " + str(end-start) + " seconds")
start = end

print("Storing as Sky Coordinates...")
coord_1 = SkyCoord(ra= cut_pairs_df['RA_1']*u.degree,dec = cut_pairs_df['DEC_1']*u.degree)
coord_2 = SkyCoord(ra= cut_pairs_df['RA_2']*u.degree,dec = cut_pairs_df['DEC_2']*u.degree)

end = time.time()
print("Done in " + str(end-start) + " seconds")
start = end


Appending Redshifts of Pair to Dataframe...
Done in 49.7289919853 seconds
Computing Average Redshift...
Done in 0.406784057617 seconds
Computing Transverse Comoving Distance at this redshift...
Done in 12.0777349472 seconds
Appending (RA,DEC) to Dataframe...
Done in 1.44557404518 seconds
Storing as Sky Coordinates...
Done in 144.226296902 seconds


In [12]:
cut_pairs_df['DEL_THETA'] = coord_1.separation(coord_2).radian


In [13]:
cut_pairs_df

,index,galaxy_index_1,galaxy_index_2,SEP_LOS,Z1,Z2,COMOV_TRV,RA_1,RA_2,DEC_1,DEC_2,DEL_THETA
0,4,75511,85064,214.586205,0.626342,0.696754,2462.151146,347.544741,347.166787,-54.570891,-54.593484,0.003843
1,13,8051,12914,153.027008,0.284317,0.324905,1248.790345,28.070120,28.024114,-55.036491,-55.246376,0.003692
2,31,31255,49275,264.735699,0.438714,0.516440,1869.097134,356.452801,356.654295,-53.411940,-53.255509,0.003444
3,35,46905,66468,228.572248,0.507206,0.576971,2084.607330,7.717847,7.959897,-58.326549,-58.304697,0.002251
4,39,57206,75342,256.394069,0.544730,0.625037,2223.010274,29.264691,29.066008,-52.838539,-53.174458,0.006223
5,45,12362,20305,215.602112,0.320597,0.379297,1417.506292,340.378069,340.215354,-56.275591,-56.603689,0.005938
6,46,17937,29386,236.474275,0.363056,0.429204,1584.860395,352.615729,352.918837,-54.986434,-54.945155,0.003121
7,53,41851,54077,161.646910,0.485758,0.534151,1978.304092,350.268080,350.138262,-52.777030,-52.650077,0.002606
8,58,35115,48872,195.580257,0.457262,0.514981,1898.123945,10.928696,11.105506,-53.213454,-53.545885,0.006087
9,63,76843,87950,236.386338,0.636009,0.714213,2503.313157,30.928243,30.704491,-52.651084,-52.387460,0.005179
